In [1]:
import pandas as pd
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash 
from dash import dcc
import numpy as np
from dash import html
from dash.dependencies import Input, Output, State
import requests as re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

import finnhub         #(anaconda prompt) => pip install finnhub-python 

# Aufgabe 1:

## Datenbeschaffung - "Company Characteristics":
 
Bei der Beschaffung von Daten mittels API requests orientiere ich mich am Kapitel "Online Data" aus dem Skript.
Um die vollständigen Daten für den Unternehmenssteckbrief zu erhalten sind Informationen aus zwei verschiedenen "API docs" nötig. Jegliche Daten aus dem folgenden Abschnitt entnehme ich Financialmodelingprep.


#### Vorgehen:
Bei der Datenbeschaffung mittels API Calls wird die URL passend zusammengesetzt um die gewünschte "response" zu erhalten.

- Zuerst werden die Informationen aus "dowjones_constituent" im Dataframe "dowjones_data" gespeichert.

- In der darauffolgenden Zelle werden alle Daten, die aus den jeweiligen Profilen der Unternehmen hervorgehen, in einem Dataframe "company_data"    zusammengefasst. Das zusammenfassen realisiere ich mittels einer For-Schleife, die ebenfalls einen Fehler auswirft sollte eine Anfrage fehlschlagen.

- Mit Hilfe eines merges auf "Symbol" werden die Daten von "dowjones_data" und "company_data" miteinander verknüpt. 
  Im letzten Schritt werden alle unrelevanten Daten ausgegliedert.
  Es bleibt also ein Dataframe (dj_data), welcher alle nötigen Informationen für Aufgabe 1 enthält.

In [2]:
# Liste aller Dow Jones Unternehmen:

api_key = '069ebbce7fc193df56ca24c83b30662e'
base_url = 'https://financialmodelingprep.com/api/v3'
filling = '/dowjones_constituent'
params = {'apikey': api_key}

response = re.get(base_url + filling, params = params)   #die Antwort von der request wird in der variable "response" gespeichert

dowjones_data = pd.DataFrame.from_dict(response.json())  #verwandeln der response to json in ein Dataframe

In [3]:
#Profile aller Dow Jones Unternehmen:

company_profile = pd.DataFrame()

for symbol in dowjones_data.symbol: 
    profile_url = base_url + '/profile/' + symbol
    profile_res = re.get(profile_url, params= params)
    if profile_res.status_code != 200:
        print('request failed')
        break
        
    profile_data = pd.DataFrame.from_dict(profile_res.json())
    company_profile = company_profile.append(profile_data)
    
company_profile = company_profile.reset_index(drop=True) # anpassung der Zeilennummerieung


In [4]:
# Zusammenführen von dowjones_data und company_profile zu einem Dataframe(dj_data):

dj_data = dowjones_data.merge(company_profile, on ='symbol')
dj_data = dj_data[['companyName','symbol','headQuarter', 'fullTimeEmployees','ceo','image']] # nur relevante Daten im Dataframe behalten

dj_data = dj_data.rename(columns ={'companyName' : 'name'})  #vereinfachtung
dj_data = dj_data.rename(columns ={'image' : 'companyLogo'}) #vereinfachung


## Datenbeschaffung - "Current Headlines":

Mit dem Code der nächsten Zelle besorge ich mir die nötigen Daten für drei Unternehmensbezogene aktuelle Artikel.
Da Financialmodelingprep keine geeigneten "API-Calls" für Artikel bereitstellt habe ich mich für die alternative "Finnhub" entschieden.
Finnhub liefert die Möglichkeit den Veröffentlichungszeitraum der Artikel anzugeben und gibt nur Unternehmensspezifische Artikel zurück.
- Mehr Informationen zu Finnhub: https://finnhub.io/

#### Vorgehen:

Zuerst ist der import von "Finnhub" nötig. Finnhub vereinfacht den Prozess des Verbindungsaufbaus und der Artikelbeschaffung, denn die Methode "company_news()" übernimmt die Anfragen für die Unternehmensspezifischen Artikel. 
Daraufhin werden mit jeder Schleifeniteration die Daten des aufgerufenen Symbols im Dataframe "news_data" gespeichert.
Die Anzahl habe ich, in Codezeile 7, auf drei aktuelle Headlines/URls begrenzt.


In [5]:
# Der folgende Code ist wichtig da das Dashboard die Unternehmensnamen zur Ausführung benötigt. 
# Dadurch werden die Unternehmen im Dataframe "news_data" nicht anhand ihres Symbols sonder mittels namen gespeichert.
company_symbols = []
company_names = []
for x in range(len(dj_data)):
    company_symbols.append(dj_data.iloc[x]['symbol'])
    company_names.append(dj_data.iloc[x]['name'])

In [6]:
finnhub_client = finnhub.Client(api_key="cc3o6kqad3i9vsk3ump0")                      # Client aufbau (übernimmt verbindung)
today = date.today()  
news_data = pd.DataFrame()                                                           # erstellen des News Dataframes
counter = 0
for symbol in company_symbols:                                                       # Iteriert über alle Symbole, denn.company_news() braucht das jeweilige Symbol als Input
    finn = finnhub_client.company_news(symbol, _from = "2022-08-01", to = today)     # company_news Methode von Finnhub übernimmt die request Anfragen für das jeweilige Symbol in einem bestimmten Zeitraum
    news_data[company_names[counter]]= [finn[0]['headline'], finn[0]['url'], finn[1]['headline'], finn[1]['url'], finn[2]['headline'], finn[2]['url']]
    counter+=1
# Zeilenindex 0,2,4 sind die jewiligen Headlines der drei Artikl
# Zeilenindex 1,3,5 sind die jeweiligen URLs

### Ausgabe der Headlines:
- hierfür habe ich mir für jede Headline und URL eine passende Getter Methode erstellt.
- die latest_news Methode wird dann in der Dasboard Zelle aufgerufen.

In [7]:
# Getter Methoden für Headlines und Urls

def get_headline_one(comp_name):
    return news_data[comp_name][0]

def get_url_one(comp_name):
    return news_data[comp_name][1]

def get_headline_two(comp_name):
    return news_data[comp_name][2]

def get_url_two(comp_name):
    return news_data[comp_name][3]

def get_headline_three(comp_name):
    return news_data[comp_name][4]

def get_url_three(comp_name):
    return news_data[comp_name][5]

In [8]:
def latest_news(comp_name):
# ich übergebe den Namen der Headline und den passenden Link, zudem stelle ich mit (Target...) sicher das beim öffnen ein neuer Tab geöffnet wird    
    return html.Div([
            html.P([
                html.A(get_headline_one(comp_name), href=get_url_one(comp_name), target='_blank')  
            ]),
            html.P([
                html.A(get_headline_two(comp_name), href=get_url_two(comp_name), target='_blank')
            ]),
            html.P([
                html.A(get_headline_three(comp_name), href= get_url_three(comp_name), target='_blank')
            ])
        ])

## Hilfsfunktion für Daten aus Aufgabe 1:
Die folgende Zelle enthält eine Funktion get_informationen welche die relevanten Informationen des ausgewählten Unternehmens als Array zurück gibt, sodass diese auf dem Dashboard ausgeworfen werden können.

In [9]:
# Hilfsfunktion für Datenausgabe auf Dashboard:

def get_information(df, comp_name):
    info_df = dj_data[dj_data.name == comp_name]
    
    logo = info_df['companyLogo'].unique()[0]
    name = info_df['name'].unique()[0]
    symbol = info_df['symbol'].unique()[0]
    headQuarter = info_df['headQuarter'].unique()[0]
    employees = info_df['fullTimeEmployees'].unique()[0]
    ceo = info_df['ceo'].unique()[0]
    
    return logo, name, symbol, headQuarter, employees, ceo

# Aufgabe 2:

## Datenbeschaffung - "Close Prices":

Das Vorgehen der Datenbeschaffung bleibt gleich wie bei "Datenbeschaffung - Company Characteristics". 
Die nötigen Daten werden von Financialmodelingprep bereitgestellt.
Es werden die "historical-full-prices" der verschiedenen Dow Jones Unternehmen per API-Call requested und in einem Dataframe "close_prices" gespeichert.



In [10]:
close_prices = pd.DataFrame()
cp_params = {'from' : '2017-01-01', 'to' : '2021-12-31' , 'apikey': api_key} # Data from 2017 to the end of 2021

for symbol in dowjones_data.symbol:
    cp_url = base_url + '/historical-price-full/' + symbol 
    cp_res = re.get(cp_url, params = cp_params)
    
    if cp_res.status_code != 200:
        print('something went wrong')
        break
    
    cp_data = pd.DataFrame.from_dict(cp_res.json())
    cp_data = pd.DataFrame.from_records(cp_data.historical)
    cp_data['symbol'] = symbol                                               # wichtig für den Merge in der nächsten Codezelle
    
    close_prices = close_prices.append(cp_data)



## Anpassungen des Dataframes - "Close Prices":

Um das Dataframe (close_prices) zu vervollständigen wird einerseits ein Merge mit dowjones_data durchgeführt, des Weiteren der wird der Index durch das "date" dargestellt und zuletzt werden die Zeilen umgekehrt ausgegeben.


In [11]:
close_prices = dj_data[['name', 'symbol']].merge(close_prices, on= 'symbol')

close_prices_date_attribute = close_prices    # Dataframe mit 'date' als Spalte in neuer Variable gespeichert.

close_prices = close_prices.set_index('date') # Das Datum als neuer Index, für die folgenden Berechnungen wichtig (auch für Codezeile 7 wichtig)

close_prices = close_prices[::-1]             #"reversed Index date" (für die RSI und SMA Funktionen wichtig)

## Funktionen zur Berechnung des RSI and SMA:

#### Relative Strength Index (RSI): 
- die Funktion "calculate_rsi" berechnet für jedes Unternehmen den zugehörigen RSI.
- Bei der Berechnung des RSI habe ich mich für die Variante mit dem "Simple Moving Average (SMA)" entschieden
     (eine andere mögliche Variante ist die Brechnung mit dem "Exponential Moving Average (EWM)").

In [39]:
def calculate_rsi(data, period = 14):                        # "default value of 14 days"
   
    rsi_data = pd.DataFrame()
    temp_rsi = pd.DataFrame()    
    
    for sym in data.symbol.unique():  
        temp_rsi = data[data.symbol==sym]
    
        diff = temp_rsi.close.diff().dropna()                # unterschied von "close prices" (daily)
                                                   
        up = 0 * diff
        down = 0 * diff
                                                  
        up[diff > 0] = diff[diff > 0]                        # "up" ist gleich zu positiver differenz
        down[diff < 0] = diff[diff < 0]                      # "down" ist gleich zu negativer differenz
    
        up_avg   = up.rolling(window = period).mean()        # Berechnung des rolling means
        down_avg = abs(down.rolling(window = period).mean()) # Bei negativen Werten wird der absolute Wert genommen
    
        rs = abs(up_avg/down_avg)                            # berechnung der "relativ strength"
       
        temp_rsi['rsi'] = 100 - (100 / (1 + rs))             #rsi berechnung und speicherung in temporärem Dataframe
    
        rsi_data = rsi_data.append(temp_rsi)
    
    return rsi_data

pd.options.mode.chained_assignment = None                    # default='warn', verhindert unnötige Warnung

rsi = calculate_rsi(close_prices, 14)                        #Methodenaufruf mit close_prices als Argument   
rsi

,name,symbol,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,rsi
date,,,,,,,,,,,,,,,
2017-01-03,The Procter & Gamble Company,PG,83.880,84.3900,83.5000,84.20,69.928970,8794071,8792700,0.320,0.381500,84.03,"January 03, 17",0.003815,NaN
2017-01-04,The Procter & Gamble Company,PG,84.450,84.6016,84.2354,84.50,70.178123,8096711,8096700,0.050,0.059207,84.44,"January 04, 17",0.000592,NaN
2017-01-05,The Procter & Gamble Company,PG,84.410,85.4274,84.3800,85.06,70.643234,6852540,6852400,0.650,0.770050,84.95,"January 05, 17",0.007700,NaN
2017-01-06,The Procter & Gamble Company,PG,84.941,85.2500,84.6200,85.03,70.618294,4800046,4799900,0.089,0.104780,85.04,"January 06, 17",0.001048,NaN
2017-01-09,The Procter & Gamble Company,PG,84.060,84.6600,83.7500,84.40,70.095093,11000778,11000600,0.340,0.404470,84.30,"January 09, 17",0.004045,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,Amgen Inc.,AMGN,223.810,225.4600,222.6700,225.17,214.345154,1763325,1763200,1.360,0.607660,224.56,"December 27, 21",0.006077,80.514158
2021-12-28,Amgen Inc.,AMGN,225.170,226.9800,225.0500,225.77,214.916321,1304468,1304500,0.600,0.266470,225.85,"December 28, 21",0.002665,79.004416
2021-12-29,Amgen Inc.,AMGN,225.582,228.2100,224.9900,227.60,216.658325,1389582,1389400,2.018,0.894570,227.29,"December 29, 21",0.008946,78.679168


#### Simple Moving Average (SMA):
- Ich berechne den SMA mithilfe der .rolling() Methode. Auf diese Sequenz von werten wird dann die .mean() Methode angewendet, wodurch "Moving Averages" enstehen.
- Information zu .rolling(): https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html


In [13]:
def calculate_sma(data):
    
    sma_data = pd.DataFrame()
    temp_sma = pd.DataFrame()
    
    for sym in data.symbol.unique():
        temp_sma = data[data.symbol==sym]                             # immer nur die Daten von einem Unternehmen im temporärem Dataframe
        
        temp_sma['SMA(slow)']=temp_sma['close'].rolling(200).mean()   # in den Klammern der rolling() Methode steht die Länge der Periode (Zeitfenster)
        temp_sma['SMA(fast)']=temp_sma['close'].rolling(50).mean()  
        
        sma_data = sma_data.append(temp_sma)
           
    return sma_data

pd.options.mode.chained_assignment = None                              # default='warn', verhindert unnötige Warnung

smas = calculate_sma(close_prices)                                     # Methodenaufruf mit close_prices als Argument
smas = smas[['SMA(fast)','SMA(slow)']]                                 # nur die SMAs speichern



## Datenzusammenführung von RSI und SMA:

Die folgende Codezelle fügt den "RSI" und den "SMAs" Dataframe zusammen. Da die Indices von "RSI" und "SMAs" Identisch sind bietet sich die .concat() Funktion an. Die .concat() Funktion nutzt dann die identischen Indices für ein Mapping zwischen den Objekten.

In [14]:
rsi_sma_close = pd.concat([rsi, smas], axis = 1)                                             #Zusammenführen

rsi_sma_close = rsi_sma_close[['symbol', 'name','close','rsi','SMA(fast)','SMA(slow)']]      #nur relevante Spalten auswählen

# Erstellen der Graphen:
Die folgenden zwei Funktionen geben die gewünschten Graphen(RSI und Trendline/Sma Graphen) aus "Aufgabe 2" zurück.

#### Vorgehen:
Das Vorgehen ist bei beiden Funktionen (Graphen) nahezu Identisch.
- Zuerst wird geprüft ob der Name des Unternehmens im Dataframe vorhanden ist. Daraufhin werden die Daten des gewollten Unternehmens in ein Temporäres Dataframe geladen. Es werden also nur die Daten vom gewünschten Unternehmen verwendet.
- Die For-Schleife liefert die passenden Daten (Spalten) woduch passende Traces (.add_trace()) hinzugefügt werden.
- Bei der Erstellung der "Buttons" und des "Rangesliders" habe ich mich auch an Übung 10 orientiert. Des Weiteren können die Buttons über ein Dropdown Menü erreicht werden.
- Der Titel und die Achsenbeschriftungen werden über die "update Methoden" definiert.

In [15]:
graph_data = rsi_sma_close # Dataframe mit allen nötigen Daten.

### Funktion für den "SMA-Graphen"/"Trendline-Graphen": 

Als Trendlinie habe ich die Verbindung zwischen den "Close Prices" verstanden.

In [16]:
def smas_graph(comp_name):

    close_prices_date_attribute.sort_values('date', inplace=True)      # nötig um das Datum auf der X-Achse richtig anzuzeigen

    SMAs_fig = go.Figure()
    
    temp_graph_data = pd.DataFrame()
    
    if comp_name in graph_data.name.unique():
        temp_graph_data = graph_data[graph_data.name == comp_name]
        
        temp_graph_data =temp_graph_data.rename(columns = {'close': 'Trendline'}) 
        
        for col in ['Trendline', 'SMA(slow)', 'SMA(fast)']:
            SMAs_fig.add_trace(go.Scatter(x = temp_graph_data.index, y = temp_graph_data[col], name= f"{col}", visible=False))

        SMAs_fig['data'][0]['showlegend']=True # Legende soll auch bei nur einem Graph zu sehen sein

    # Button Erstellung:
    std_buttons = [dict(
        method= 'restyle', label= 'reset',
        args= [{'visible': [False, False, False]}]
    ),
                    dict(
        method= 'restyle', label= 'Trendline',
        args= [{'visible': [True, False, False]}]
    ),
                   dict(
        method= 'restyle', label= 'SMA(slow)(200)',
        args= [{'visible': [True, True, False]}]
    ),           
                   dict(
        method= 'restyle', label= 'SMA(fast)(50)',
        args= [{'visible': [True, False, True]}]
    ),
                  dict(
        method= 'restyle', label= 'All',
        args= [{'visible': [True, True, True]}]
    )]
    
    
    
    
    
    SMAs_fig.update_layout(updatemenus=[dict(
                type="dropdown",
                buttons= std_buttons)],
                title="Company Trendline and Simple Moving Averages:",
                legend_title="Legend: ",
                font_family = "Arial Black",
                
                  )

    SMAs_fig.update_layout(
       xaxis=dict(
           rangeslider=dict(visible=True),
           type="date",
            range=[close_prices_date_attribute.date.iloc[4], close_prices_date_attribute.date.iloc[-4]]      # close_prices_not_reversed_date date für passende x achse
        )
             
    )
    SMAs_fig.update_xaxes(
            title_text = "Date",
            title_font = {"size": 20, "family": "Arial "},
            title_standoff = 20,
            )

    SMAs_fig.update_yaxes(
            title_text = "Monetary Units",
            title_font = {"size" : 15, "family" : "Arial"},
            title_standoff = 5)


    return SMAs_fig

### Funktion für den "RSI-Graphen":


##### Unterschied:
Zuerst wird der Trace für den RSI Graphen separat erstellt und die For_schleife bildet dann, mit zwei Iterationen, die Traces für die 30/70 Prozentlinien. 

In [17]:
def rsi_graph(comp_name):

    close_prices_date_attribute.sort_values('date', inplace=True)
    
    rsi_fig = go.Figure()
    
    if comp_name in graph_data.name.unique():
        temp_graph_data = graph_data[graph_data.name == comp_name]
    
        rsi_fig.add_trace(go.Scatter(x = temp_graph_data.index, y = temp_graph_data.rsi, name= f"{'Rsi'}", visible = False))

        for num in [70, 30]:
                rsi_fig.add_trace(go.Scatter(x= temp_graph_data.index, y=[num] * len(temp_graph_data.index), name= f"{num}", visible = False))

        rsi_fig['data'][0]['showlegend']=True   # Legende soll auch bei nur einem Graph zu sehen sein.
        
    
    
    
    std_buttons = [dict(
        method= 'restyle', label= 'reset',
        args= [{'visible': [False, False, False]}]
    ),             dict(
        method= 'restyle', label= 'RSI',
        args= [{'visible': [True, False, False]}]
    ),
                   dict(
        method= 'restyle', label= '70 Percent',
        args= [{'visible': [True, True, False]}]
    ),           
                   dict(
        method= 'restyle', label= '30 Percent',
        args= [{'visible': [True, False, True]}]
    ),
                  dict(
        method= 'restyle', label= 'All',
        args= [{'visible': [True, True, True]}]
    )]
    
    
    
    
    rsi_fig.update_layout(updatemenus=[dict(
                        type="dropdown",
                        buttons= std_buttons)],   
                        title="Company RSI Graph:",
                        legend_title="Legend: ",
                        font_family = "Arial Black",
                          
                  )

    rsi_fig.update_layout(
       xaxis=dict(
          rangeslider=dict(visible=True),
            type="date",
            range=[close_prices_date_attribute.date.iloc[4], close_prices_date_attribute.date.iloc[-4]]   # close_prices_date_attribute date für passende x achse  
       )
    
    )
    rsi_fig.update_xaxes(
            title_text = "Date",
            title_font = {"size": 20, "family" : "Arial"},
            title_standoff = 5)

    rsi_fig.update_yaxes(
            title_text = "Monetary Units",
            title_font = {"size": 15, "family" : "Arial"},
            title_standoff = 20)


    return rsi_fig

# Trading-Strategien:
- Erstellung eines extra Dataframes (trading_data) mit zwei neuen Spalten.

In [18]:
trading_data = rsi_sma_close
trading_data['balance'] = 10000   # balance Spalte
trading_data['stocks'] = 0        # stock Spalte

#### Hilfsmethoden für die folgenden Trading Strategien:

In [19]:
def sell(data, x):     # balance und stocks nach Verkauf von Aktien anpassen
    data.balance[x] = data.balance[x-1] + (data.close[x] * 10)
    data.stocks[x] = data.stocks[x-1] - 10 

    
def buy(data, x):      # balance und stocks nach Kauf von Aktien anpassen
        data.balance[x] = (data.balance[x] - (10 * data.close[x]))
        data.stocks[x] = data.stocks[x-1] + 10

        
def sell_all(data):    # balance und stocks anpassen nachdem alle Aktien aus dem Depot verkauft wurden
    data.balance[len(data.index)-1] = data.balance[len(data.index)-1] + (data.stocks[len(data.index)-1] * data.close[len(data.index)-1])
    data.stocks[len(data.index)-1] = 0

## Trading-Strategie 1 - "RSI Trading":

#### Bedingungen:

- Der startbetrag beträgt 10.000 Monetäre Einheiten.
- Sobald "RSI > 70 (Overbought)"  werden 10 Aktien aus dem Portfolio verkauft.
- Sobald "RSI < 30 (Oversold)" werden 10 Aktien gekauft.
- Es wird immer zum Tagesaktuellen "Close Prices" gehandelt. 
- Eine Kauf-Transaktion findet nur statt wenn genug monetäre Einheiten bereit stehen um 10 Aktien zu kaufen. Eine Verkauf-Transaktion findet nur statt wenn auch genug Aktien im Depot vorhanden sind.
- zum letzten Handeltag im Dataframe (Ende 2021) werden alle Aktien verkauft und mit dem Restkontostand addiert.

In [20]:
def rsi_trading(comp_name):
    x = 0
    
    if comp_name in trading_data.name.unique():
        temp_trading = trading_data[trading_data.name == comp_name]            # nur die Daten für das ausgewählte Unternehmen
        temp_trading = temp_trading.reset_index()
        
        for index in temp_trading.index:                                       # iteriert über den den ganzen Dataframe
            
            if x > 0:                                                          # nötig da bei x = 0 ist [x-1] nicht möglich
                temp_trading.stocks[x] = temp_trading.stocks[x-1]              # stock übernimmt Anzahl vom Vortag
                temp_trading.balance[x] = temp_trading.balance[x-1]            # balance übernimmt anzahl vom Vortag 
                
             
            if temp_trading.rsi[x] < 30 and temp_trading.rsi[x] != 'NaN':
                if temp_trading.balance[x] > (10 * temp_trading.close[x]):
                    
                    buy(temp_trading, x)                                       # Methodenaufruf 
                    
                                  
            elif temp_trading.rsi[x] > 70:
                if temp_trading.stocks[x - 1] >= 10:
                    
                    sell(temp_trading, x)                                      # Methodenaufruf 
                         
            x += 1
            
        sell_all(temp_trading)                                                 # Methodenaufruf 
            
    return temp_trading

## Trading Strategie 2 - "SMA Trading":

#### Bedingungen:
- Der Startbetrag beträgt 10.000 Monetäre Einheiten.
- Sobald SMA(fast) größer als SMA(slow) wird, werden 10 Aktien gekauft.
- Sobald SMA(slow) größer als SMA(fast wird, werden 10 Aktien gekauft. Also wird eine Transaktion nur bei einem wechsel des Größenerhältnisses zwischen SMA(slow) und SMA(fast) ausgeführt.

- Es wird immer zum Tagesaktuellen "Close Prices" gehandelt. 
- Eine Kauf-Transaktion findet nur statt wenn genug monetäre Einheiten bereit stehen um 10 Aktien zu kaufen. Eine Verkauf-Transaktion findet nur statt wenn auch genug Aktien im Depot vorhanden sind.
- zum letzten Handeltag im Dataframe (Ende 2021) werden alle Aktien verkauft und mit dem Restkontostand addiert.

In [21]:
def sma_trading(comp_name):
    smas_temp_trading = pd.DataFrame()
    
    x = 0
    
    if comp_name in trading_data.name.unique():
        smas_temp_trading = trading_data[trading_data.name == comp_name]                 # nur die Daten für das ausgewählte Unternehmen
        
        smas_temp_trading = smas_temp_trading.reset_index()
        smas_temp_trading = smas_temp_trading.rename(columns ={'SMA(fast)' : 'sma_fast'}) 
        smas_temp_trading = smas_temp_trading.rename(columns ={'SMA(slow)' : 'sma_slow'}) 
        
        for index in smas_temp_trading.index:                                            # iteriert über den den ganzen Dataframe
            
            if x > 0:                                                                    # nötig da bei x = 0 ist [x-1] nicht möglich
                smas_temp_trading.stocks[x] = smas_temp_trading.stocks[x-1]              # stock übernimmt Anzahl vom Vortag
                smas_temp_trading.balance[x] = smas_temp_trading.balance[x-1]            # balance übernimmt Anzahl vom Vortag 
                
            
            # folgende if Bedingung schaut ob aktull (fast > slow) gilt und ob am Vortag (slow > fast) galt. Außerdem muss sma_slow einen Wert zugeordnet haben
            if smas_temp_trading.sma_fast[x] > smas_temp_trading.sma_slow[x] and smas_temp_trading.sma_fast[x-1] < smas_temp_trading.sma_slow[x-1] and smas_temp_trading.sma_slow[x] != 'NaN':
                if smas_temp_trading.balance[x] > (10 * smas_temp_trading.close[x]):
                    
                    buy(smas_temp_trading, x)                                            # Hilfsmethoden Aufruf
                    
            
            elif smas_temp_trading.sma_slow[x] > smas_temp_trading.sma_fast[x] and smas_temp_trading.sma_slow[x-1] < smas_temp_trading.sma_fast[x-1]:
                if smas_temp_trading.stocks[x - 1] >= 10:
                    
                    sell(smas_temp_trading, x)                                           # Hilfsmethoden Aufruf
                                      
            x += 1                                                                       # index +1

        sell_all(smas_temp_trading)    
    
    return smas_temp_trading

## Trading Strategie 3 - "Buy and Hold":
 
#### Bedingungen:
- Der Startbetrag beträgt 10.000 Monetäre Einheiten.
- Am ersten Trading Tag werden möglichst viele Aktien gekauft und am letzten Trading Tag werden diese wieder verkauft.
- Der folgende Code beachtet dabei quasi also nur die erste und letzte Zeile.

In [22]:
def buy_hold(comp_name):
    if comp_name in trading_data.name.unique():
            bh_temp_trading = trading_data[trading_data.name == comp_name]
            bh_temp_trading = bh_temp_trading.reset_index()
             
            y = (10000//bh_temp_trading.close[0])   # '//' für gerundeten Wert | gibt Anzahl für mögliche Aktienkäufe 
            
            bh_temp_trading.balance[0] = bh_temp_trading.balance[0] - (bh_temp_trading.close[0] * y)
            #rest_balance = bh_temp_trading.balance[0]
            bh_temp_trading.stocks[0] = y
            
            
            bh_temp_trading.balance[len(bh_temp_trading.index)-1] = bh_temp_trading.balance[0] + (bh_temp_trading.close[len(bh_temp_trading.index)-1] * y)
            bh_temp_trading.stocks[len(bh_temp_trading.index)-1] = 0
            
    return bh_temp_trading

### Dataframe Erstellung für Endresultate der Trading-Strategien:
- Für jedes Unternehmen (index) werden die Endergebnisse der Trading-Strategien gespeichert.


In [23]:
compare_trading = pd.DataFrame()
compare_trading['name'] = trading_data.name.unique()    
compare_trading = compare_trading.set_index('name')

# Spalten erstellen:
compare_trading['rsi_strategy'] = 0          
compare_trading['sma_strategy'] = 0
compare_trading['bh_strategy'] = 0


# jede Zeile/Unternehmen nacheinander mit Werten füllen:
for comp_name in trading_data.name.unique():
    rsi_last_balance = rsi_trading(comp_name)                                     # rsi Dataframe übergeben
    rsi_last_balance = rsi_last_balance.balance[len(rsi_last_balance.index)-1]    # letzten Wert aus rsi Dataframe speichern
    
    sma_last_balance = sma_trading(comp_name)
    sma_last_balance = sma_last_balance.balance[len(sma_last_balance.index)-1]    #...(siehe rsi)
    
    bh_last_balance = buy_hold(comp_name)
    bh_last_balance = bh_last_balance.balance[len(bh_last_balance.index)-1]       #...
    
    
    compare_trading.at[comp_name,'rsi_strategy'] = rsi_last_balance               # Werte an passende Zeile im compare Dataframe übergeben
    compare_trading.at[comp_name,'sma_strategy'] = sma_last_balance
    compare_trading.at[comp_name,'bh_strategy'] = bh_last_balance   

## Diagrammerstellung zum Vergleichen der Trading-Strategien:
- um die Trading-Stragien bestmöglich miteinander zu vergleichen habe ich mich für ein Balkendiagramm entschieden 

In [24]:
template = dict(
    layout=go.Layout(title_font=dict(family="Arial Black", size=20))
)

In [25]:
def trading_bar(comp_name):
        
    df = pd.DataFrame({'Trading Strategies': ['RSI Trading','SMA Trading', 'Buy and Hold Trading'], 'Result in Monetary units': [compare_trading.rsi_strategy[comp_name], compare_trading.sma_strategy[comp_name], compare_trading.bh_strategy[comp_name] ]})
    fig = px.bar(df, x ='Trading Strategies', y ='Result in Monetary units',text = 'Trading Strategies', title ='Comparison Of Different Trading Strategies For The Respective Company:')
    
    fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=10000, y1=10000, yref="y"
    )
    
    fig.update_layout(template=template)
   
    return fig

## Zusammenhangsanalyse:

Im folgendem Teil wird der Zusammenhang zwischen dem "RSI" und den "Close Prices" betrachtet. Dabei prüfe ich, ob der RSI von "Heute" die Close Prices von "Morgen" vorhersagen kann.

#### Vorgehen:
- Ich erstelle mir eine Spalte die, die "Close Prices" um genau einen Tag verzögert betrachtet (Der "Close Price" wird um einen Tag in die Vergangenheit verschoben). So kann ich ermitteln ob der RSI von "Heute" einen Einfluss auf den "Close Price" von "Morgen" hat.

In [26]:
# Dataframe Erstellung:
lr_data = rsi_sma_close
lr_data = lr_data[['symbol', 'name','close','rsi']] 
lr_data["lagged_close_prices"] = lr_data.close.shift(periods = -1)
#lr_data

In [27]:
lr_data = lr_data.dropna()  # alle Spalten mit NaN Werten werden gelöscht

#### Erstellung des Graphen:

In [28]:
def regression(comp_name):
    
    if comp_name in lr_data.name.unique():
        temp_lr_data = lr_data[lr_data.name == comp_name]

        figg = px.scatter(
        temp_lr_data, x='rsi', y='lagged_close_prices', opacity=0.65,
        trendline='ols', trendline_color_override='red'
        )
        
        return figg

#### Berechnung des Koeffizienten:
- die Erklärung des Koeffizienten wird in der App ausgegeben.
- bei der Brechnung habe ich mich am Skript - ("SciKit Learn") orientiert.

In [29]:
def coef_lr(comp_name):
    
    if comp_name in lr_data.name.unique():
        temp_lr_data = lr_data[lr_data.name == comp_name]
        X = np.array(temp_lr_data.rsi).reshape(-1,1)
        y = np.array(temp_lr_data.lagged_close_prices).reshape(-1,1)

        linreg = LinearRegression(fit_intercept=True)
        linreg.fit(X,y)
        
        y_pred = linreg.predict(X)
        
    return linreg.coef_[0][0]

# Aufgabe 3:

## Zusammenfassen der Trading Ergebnisse auf Dow-Jones Ebene:

- dazu summiere ich alle Endresultate der verschiedenen Unternehmen je Tradingstrategie auf und erstelle daraus ein Balkendiagramm, welches für jedes Unternehmen gleich, in der App ausgeworfen wird.
- somit erhält man einen Graphen der den Dow Jones als ganzes betrachtet und dementsprechend die Trading Strategien vergleicht.

In [30]:
overall_trading_data = compare_trading  # overall Dataframe erstellen 

#### Aufsummieren der jeweiligen Endresultate:
- die Unternehmenspezifischen Endresultate für die jeweilige Strategie werden summiert.
- ich erhalte also drei Werte welche nun die Trading Strategien, bezogen auf den Dow Jones, als ganzes vergleichen.

In [31]:
def overall_balances():
    sum_rsi_trading = 0
    sum_sma_trading = 0
    sum_bh_trading = 0
    for index in range(len(overall_trading_data)):
        sum_rsi_trading += overall_trading_data.rsi_strategy[index]
        sum_sma_trading += overall_trading_data.sma_strategy[index]
        sum_bh_trading += overall_trading_data.bh_strategy[index]
        
    return sum_rsi_trading, sum_sma_trading, sum_bh_trading

#### Erstellung des Graphen:
- das Balkendiagramm wird erstellt und es wird eine gestrichelte rote Linie modelliert, welche das Startkapital representiert.

In [32]:
def overall_trading_bar():
        
    df = pd.DataFrame({'Trading Strategies': ['RSI Trading','SMA Trading', 'Buy and Hold Trading'], 'Result in Monetary units': [400916, 309753, 624089] })         # die Zahlen sind der return der Methode overall_balances(), Konstante Zahlen die für jedes Unternehmen gleich bleiben
    fig = px.bar(df, x ='Trading Strategies', y ='Result in Monetary units',text = 'Trading Strategies', title ='Comparison Of Trading Strategies Using The Sum Of The Trading Results Of All Dow-Jones Members:')
    fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=300000, y1=300000, yref="y"
    )
    
    fig.update_layout(template=template) # Layout Anpassung (Schriftart..)
   
    return fig

## Sektorenbezogen - Datenbeschaffung und Dataframeerstellung:

- zuerst wird ein neuer Dataframe aus den Unternehmens Profilen und den "Overall" Trading Ergebnissen erstellt.
  Dadurch komme ich an die passenden Sektoren zu den Unternehmen 

In [33]:
sector_df = company_profile
sector_df = sector_df.rename(columns ={'companyName' : 'name'})  

In [34]:
overall_trading_data = overall_trading_data.reset_index()

In [35]:
sector_data = overall_trading_data.merge(sector_df, on ='name')
sector_data = sector_data[['name','symbol','rsi_strategy', 'sma_strategy','bh_strategy', 'sector']]

#### Berechnung der Endresultate der Trading Strategien je Sektor:

In [36]:
sector_end_balance = pd.DataFrame()

# erstellen passender Spalten im neuen Dataframe
sector_end_balance['sector'] = ''
sector_end_balance['Rsi_Trading'] = 0          
sector_end_balance['Sma_Trading'] = 0
sector_end_balance['Buy_and_Hold_Trading'] = 0
sector_end_balance['Trading_kapital_used_in_Sector'] = 0



for sector in sector_data.sector.unique():
    
    # für jeden Sektor variablen wieder auf null setzen
    rsi_sum = 0
    sma_sum = 0
    bh_sum = 0
    kapital = 0
    
    temp_sector = sector_data[sector_data.sector == sector]  # Dataframe auf Unternehmen mit gleichem Sektor begrenzt
    temp_sector = temp_sector.reset_index()            
    
    for i in range(len(temp_sector)):
        
        # variablen mit passenden Werten füllen (pro Iteration summieren)
        rsi_sum = rsi_sum + temp_sector.rsi_strategy[i]
        sma_sum = sma_sum + temp_sector.sma_strategy[i]
        bh_sum = bh_sum + temp_sector.bh_strategy[i]
        kapital = kapital + 10000
        
    # am Ende der ersten For-Schleife die Spalten im anfangs erstellten Dataframe befüllen
    sector_end_balance.at[sector,'Rsi_Trading'] = rsi_sum
    sector_end_balance.at[sector,'Sma_Trading'] = sma_sum
    sector_end_balance.at[sector,'Buy_and_Hold_Trading'] = bh_sum
    sector_end_balance.at[sector, 'Trading_kapital_used_in_Sector'] = kapital
    
    
# Verfeinerung des Dataframes
sector_end_balance = sector_end_balance.drop('sector', axis =1)
sector_end_balance = sector_end_balance.reset_index()
sector_end_balance = sector_end_balance.rename(columns ={'index': 'sector'})

#### Balkendiagramm für den Sektorenvergleich erstellen:

In [37]:
def sector_graph():

    fig = px.bar(sector_end_balance, x = "sector", y=["Trading_kapital_used_in_Sector", "Rsi_Trading", "Sma_Trading", "Buy_and_Hold_Trading"], title="Performance Of Trading Strategies Across Sectors: ")
    fig.update_layout(template=template)
    
    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ))
    return fig

# Dashboard:

- In der folgenden Zelle wird das Dashboard erstellt. Dazu werden alle Komponenten des Notebooks passend zusammengesetzt.
- Aus ästhetischen Gründen habe ich mich für ein zentriertes Layout der App entschieden. Alle wichtigen Teile sind mit einer horizontalen Linie voneinander getrennt.

In [38]:
app = JupyterDash('Dow Jones')

app.layout = html.Div([
    html.H1('Dow Jones Dashboard', style={'font-family':'Arial Black', 'text-align':'center',} ),
    html.P([
    html.H5('Select a Dow-Jones Company:', style={'font-family':'Arial Black', 'padding-bot':'30', 'text-align':'center'}),
        dcc.Dropdown(id='company_dropdown', style={'font-family':'Arial Black', 'padding-bot':'30', 'text-align':'center'},
                options = [{'label': i, 'value': i} for i in dj_data.name.unique()],
                value = dj_data.name.unique()[0]), 
        
    ]),
    
# Aufgabe 1 in App integrieren:
    html.Div(html.Div(style = dict(
        textAlign = 'center',     # company logo nur so zentriebar
    ), children=[
        
        html.Br(),
        html.Br(),
        html.H2('Company Characteristics:' , style={'font-family': 'Arial Black'}),
        
        html.Div([
            html.Img(id='company_logo'),
        ]),
        html.Div([
            html.H3(('Company Name:'), style={'margin':'0 auto',  'margin-right':'10px',  'font-family': 'Arial Black'}),
            html.H4(id='company_name', style={'margin':'0 auto',  'margin-top': '5px', 'font-family':'Arial'}),
        ]),
        
        html.Div([
            html.H3(('Company Abbriviation:'), style={'margin':'0 auto','margin-right':'10px',  'margin-top': '25px', 'font-family': 'Arial Black'}),
            html.H4(id='company_abbriviation', style={'text-align':'center','display':'inline-block', 'margin':'0 auto', 'margin-top': '5px', 'font-family':'Arial', 'font-size':17}),
        ]),
        
        html.Div([
            html.H3(('Head Quarter Location:'), style={'margin':'0 auto','margin-right':'10px',  'margin-top': '25px', 'font-family': 'Arial Black'}),
            html.H4(id='company_head_quarter', style={'margin':'0 auto', 'margin-top': '5px', 'font-family':'Arial'}),
        ]),
        
        html.Div([
            html.H3(('Full Time Employees:'), style={'margin':'0 auto','margin-right':'10px',  'margin-top': '25px', 'font-family':'Arial Black'}),
            html.H4(id='company_full_time_employees', style={'display':'inline-block', 'margin':'0 auto', 'margin-top': '5px', 'font-family':'Arial'}),
        ]),
        
        html.Div([
            html.H3(('Companies current Ceo:'), style={'margin':'0 auto','margin-right':'10px',  'margin-top': '25px', 'font-family': 'Arial Black'}),
            html.H4(id='company_ceo', style={'margin':'0 auto', 'margin-top': '5px', 'font-family':'Arial'}),
        ]),
        
        html.H2('Latest Company-Specific News:', style={'text-align':'center','margin':'0 auto', 'margin-top': '50px','margin-bottom':'10px', 'font-family': 'Arial Black'}),
        html.Div(children=[
            html.P(id='news', style= {'text-align':'center', 'margin':'0 auto', 'font-size': '15pt', 'margin-bottom': '10px'})
        ]),
     ])),
    
    
# Aufgabe 2 in App integrieren:
        html.Br(),
        html.Br(),
        html.Br(),
        html.Hr(),
        html.H2('Stock Information Graphics:', style = {'font-family': 'Arial Black', 'text-align':'center' }),
        html.Div([
            dcc.Graph(id='smas_figure')
        
        ]),
        
        html.Br(),
        html.Div([
            dcc.Graph(id='rsi_figure')
        ]),
    
        html.Br(),
        html.Br(),
        html.Br(),
        html.Hr(),
        html.Br(),
        html.H2('Comparison Of Different Investment Strategies:', style = {'textAlign':'center', 'font-family': 'Arial Black'}), # 'textAlign':'center',
        html.Br(),
        html.H3('The following graphics are meant to illustrate and compare different trading strategies. The starting capital for each trading strategy is 10000 vulgarities, which is represented by the red dashed line. The trading period runs from 2017 until the end of 2021.' , style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.H3('"RSI Trading" - Strategy: sell when the RSI is above 70 and buy when the RSI is below 30.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.H3('"SMA Trading" - Strategy: buy when SMA(fast) becomes larger then SMA(slow) and sell when SMA(slow) becomes larger then SMA(fast)', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.H3('"Buy and Hold" - Strategy: buy for the first day of trading in 2017 and sell at the last day of trading in 2021.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.Br(),
        html.Br(),
        html.Br(),
        html.Div([
            dcc.Graph(id='c_compare_figure')
            
        ]),

# Aufgabe 3 in App integrieren:
        html.Br(),
        html.Div([
            dcc.Graph(id='overall_compare_figure')
        ]),
        
    
        html.H3('The chart above shows that on average the buy and hold strategy has by far the best performance. The RSI strategy secures second place, while the SMA strategy has the worst performance.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        
       
        html.Br(),
        html.Br(),
        html.Div([
            dcc.Graph(id='sector_compare')
        ]),
        html.H3('You have to keep in mind that only Dow Jones companies were considered in the evaluation. Furthermore, the number of participants varies in the sectors, as this determines the trading capital in the respective sector.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.Br(),
        html.Br(),
    
# Zusammenhangsanalyse in App integrieren:    
        html.Hr(),
        html.Br(),
        html.H2('Correlation Analysis:', style = {'textAlign':'center', 'font-family': 'Arial Black'}),
        html.H3('Can todays RSI predict tomorrows Close Price for the selected Company ?', style = {'textAlign':'center', 'font-family': 'Arial Black'}),
        html.H3('The following Chart is modeling the influence of an independent numerical variable (RSI) on the value of a dependent numerical variable (Future Close Price). The "Red Line" represents the regression line.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.Div([
            dcc.Graph(id='linear_r_figure')
        ]),
        html.Div([
            html.H3(('Coefficient:'), style={'textAlign':'center', 'font-family': 'Arial Black'}),
            html.H4(id='coef_lr', style={'textAlign':'center', 'font-family':'Arial','color':'red'}),
        ]),
        html.H3('In case of a positive value of the coefficient:', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.H3('if the RSI increases by one unit, the close price of the next day increases on average by the value of the coefficient.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.Br(),
        html.H3('With a negative value of the coefficient:', style = {'textAlign':'center', 'font-family': 'Arial'}), 
        html.H3(' if the RSI increases by one unit, the close price of the next day decreases on average by the value of the coefficient.', style = {'textAlign':'center', 'font-family': 'Arial'}),
        html.Br(),
        html.Br(),
        html.Br(),
])



@app.callback(
    Output('company_logo','src'),
    Output('company_name','children'),
    Output('company_abbriviation', 'children'),
    Output('company_head_quarter', 'children'),
    Output('company_full_time_employees', 'children'),
    Output('company_ceo','children' ),
    
    Output('news','children'),
    
    Output('smas_figure','figure'),
    Output('rsi_figure','figure'),
    Output('c_compare_figure', 'figure'),
    Output('overall_compare_figure','figure'),
    Output('sector_compare', 'figure'),
    
    Output('linear_r_figure','figure'),
    Output('coef_lr','children'),
    
    Input('company_dropdown', 'value')
)

def update_app(comp_name):
    
    info = get_information(dj_data, comp_name)
    
    news = latest_news(comp_name)
    
    smas_figure = smas_graph(comp_name)
    
    rsi_figure = rsi_graph(comp_name)
    
    c_compare_figure = trading_bar(comp_name)
    
    overall_compare_figure = overall_trading_bar()
    
    linear_r_figure = regression(comp_name)
    
    coef = coef_lr(comp_name)  # ausgeben des Koeffizienten
    
    sector_g = sector_graph()
    
    return info[0], info[1], info[2], info[3], info[4], info[5], news, smas_figure, rsi_figure, c_compare_figure, overall_compare_figure, sector_g, linear_r_figure, coef  
    
    
app.run_server(port='8000')


Dash app running on http://127.0.0.1:8000/
